In [122]:
import re
import json
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import pandas as pd

# Парсим HTML

Собираем курсы валют с сайта mig.kz

In [2]:
MIG_URL = "https://mig.kz/additional#main"

## Определяем данные, которые нам нужны

* Дата
* Обозначение валюты
* Название валюты
* Курс продажи
* Курс покупки

## Забираем HTML с сайта

In [3]:
html = requests.get(MIG_URL).text
html

'<!DOCTYPE html>\n<html prefix="og: http://ogp.me/ns#">\n<head>\n    <meta http-equiv="content-type" content="text/html; charset=utf-8"/>\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">\n    <meta name="theme-color" content="#0f65d8">\n    <meta name="mobile-web-app-capable" content="yes">\n            <meta name="description" content="Доводим до вашего сведения, что мы расширили список обслуживаемых валют "/>\n            <link rel="shortcut icon" href="/favicon.ico">\n    <title> Дополнительные валюты -  Пункты обмена валют «МиГ»</title>\n    <link rel="apple-touch-icon" sizes="180x180" href="/apple-touch-icon.png">\n    <link rel="icon" type="image/png" sizes="32x32" href="/favicon-32x32.png">\n    <link rel="icon" type="image/png" sizes="16x16" href="/favicon-16x16.png">\n    <link rel="manifest" href="/site.webmanifest">\n    <link rel="mask-icon" href="/safa

## Конвертируем HTML в объект библиотеки BeautifulSoup и находим нужные данне

In [9]:
"""
Сразу же выбираем div с классом informer-additional, 
так как в этом блоке содержатся все нужные нам данные
"""

soup = BeautifulSoup(html, 'lxml').find('div', 'informer-additional')

### Ищем и обрабатываем дату

In [72]:
date = soup.find('date').text.split()
date

['на', '07', 'ноября', '2020', '12:10']

In [69]:
# Создадим справочник для преобразования месяца в дате
months = {'января': '01',
          'февраля': '02',
          'марта': '03',
          'апреля': '04',
          'мая': '05',
          'июня': '06',
          'июля': '07',
          'августа': '08',
          'сентября': '09',
          'октября': '10',
          'ноября': '11',
          'декабря': '12'}

In [73]:
dt = datetime.strptime(f'{date[3]}{months[date[2]]}{date[1]}{date[4]}', '%Y%m%d%H:%M')
dt

datetime.datetime(2020, 11, 7, 12, 10)

### Идем по списку валют в таблице и сохраняем данные в словарь

In [77]:
"""
Внутри "супа" находим все строки таблицы (таблица единственная в блоке)
Итерируемся по строкам и сохраняем данные в dictionary
"""

currencies = {}

for row in soup.find_all('tr'):
    curr = row.find('td', 'currency')
    currency = curr.next_element.strip()
    currency_name = curr.find('div', 'description').next_element.strip()
    buy = row.find('td', 'buy').text
    sell = row.find('td', 'sell').text
    currencies[currency] = [currency_name, buy, sell]
    
currencies

{'USD': ['Доллар США', '432.3', '433.9'],
 'EUR': ['Евро', '511.5', '514.1'],
 'RUB': ['Российский рубль', '5.55', '5.61'],
 'KGS': ['Киргизский сом', '4.8', '5.2'],
 'GBP': ['Фунт стерлингов', '567.0', '577.0'],
 'CNY': ['Китайский юань', '63.3', '64.7'],
 'GOLD': ['Золотые слитки, 1 гр.', '26350.0', '27550.0'],
 'CHF': ['Швейцарский франк', '479.0', '489.0'],
 'HKD': ['Гонконгский доллар', '47.0', '55.0'],
 'GEL': ['Грузинский лари', '115.0', '135.0'],
 'AED': ['Дирхам ОАЭ', '103.0', '117.0'],
 'INR': ['Индийская рупия', '4.7', '5.7'],
 'CAD': ['Канадский доллар', '300.0', '330.0'],
 'MYR': ['Малайзийский ринггит', '87.0', '103.0'],
 'PLN': ['Польский злотый', '107.0', '123.0'],
 'THB': ['Тайский бат', '11.9', '13.9'],
 'TRY': ['Турецкая лира', '47.0', '57.0'],
 'UZS': ['100 узбекских сумов', '3.6', '4.0'],
 'UAH': ['Украинская гривна', '13.3', '15.3'],
 'CZK': ['Чешская крона', '18.5', '20.5'],
 'ILS': ['Израильский шекель', '105.0', '125.0'],
 'KRW': ['100 южнокорейских вон', '31.0

### Конвертируем словарь в датафрейм pandas

In [82]:
df = pd.DataFrame.from_dict(currencies, orient='index', columns=['currency_name', 'buy', 'sell'])
df['dt'] = dt
df

,currency_name,buy,sell,dt
USD,Доллар США,432.3,433.9,2020-11-07 12:10:00
EUR,Евро,511.5,514.1,2020-11-07 12:10:00
RUB,Российский рубль,5.55,5.61,2020-11-07 12:10:00
KGS,Киргизский сом,4.8,5.2,2020-11-07 12:10:00
GBP,Фунт стерлингов,567.0,577.0,2020-11-07 12:10:00
CNY,Китайский юань,63.3,64.7,2020-11-07 12:10:00
GOLD,"Золотые слитки, 1 гр.",26350.0,27550.0,2020-11-07 12:10:00
CHF,Швейцарский франк,479.0,489.0,2020-11-07 12:10:00
HKD,Гонконгский доллар,47.0,55.0,2020-11-07 12:10:00
GEL,Грузинский лари,115.0,135.0,2020-11-07 12:10:00


# Парсим API с одностраничных приложений (SPA)

Соберем информацию о товарах в ресторане на chocofood.kz

In [86]:
CHOCOFOOD_URL = "https://chocofood.kz/ru/almaty/ginger-sushi/menu/"

## Определяем данные, которые нам нужны

* Категория
* Название
* Описание
* Цена
* Картинка

## Забираем HTML с сайта

In [88]:
html = requests.get(CHOCOFOOD_URL).text
html

'<!doctype html>\n<html lang="en" data-n-head="%7B%22lang%22:%7B%221%22:%22en%22%7D%7D">\n  <head >\n    <title>Быстрый заказ еды из ресторанов в Казахстане - Chocofood.kz</title><meta data-n-head="1" charset="utf-8"><meta data-n-head="1" name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, minimum-scale=1"><meta data-n-head="1" data-hid="description" name="description" content="Chocofood.kz - это быстрый и удобный сервис заказа еды из ресторанов Алматы с возможностью оплаты кредитными картами и электронными деньгами."><meta data-n-head="1" data-hid="keywords" name="keywords" content="chocofood.kz, доставка еды, заказ еды, доставка пиццы, заказ еды на дом, доставка суши, доставка обедов"><meta data-n-head="1" name="author" content=""><meta data-n-head="1" name="yandex-verification" content="652c7d5b469c3982"><meta data-n-head="1" name="mailru-verification" content="8eab1f15a7a3a289"><meta data-n-head="1" name="theme-color" content="#d0021b"><meta data-n-head="

## Сайт одностраничный (мало HTML и много JS), ищем API

In [89]:
CATEGORIES_API = "https://chocofood.kz/debug-api/ginger-sushi/foodtypes/new/?slug=ginger-sushi&city_slug=almaty&image_size=xxxhdpi"

In [90]:
FOOD_API = "https://chocofood.kz/debug-api/ginger-sushi/food_list/modification/?slug=ginger-sushi&city_slug=almaty&image_size=xxxhdpi"

## Собираем данные

### Собираем категории

In [97]:
categories = requests.get(CATEGORIES_API).text
categories

'[{"id":69813,"title":"Роллы","logo":null,"description":"","restaurant":25,"parent":0,"position":1,"is_active":1,"food_count":40,"is_alcohol":false},{"id":69812,"title":"Роллы теплые","logo":null,"description":"","restaurant":25,"parent":0,"position":2,"is_active":1,"food_count":12,"is_alcohol":false},{"id":69814,"title":"Ассорти","logo":null,"description":"","restaurant":25,"parent":0,"position":3,"is_active":1,"food_count":11,"is_alcohol":false},{"id":69817,"title":"Нигири суши","logo":null,"description":"","restaurant":25,"parent":0,"position":4,"is_active":1,"food_count":5,"is_alcohol":false},{"id":69811,"title":"Гункан суши","logo":null,"description":"","restaurant":25,"parent":0,"position":5,"is_active":1,"food_count":11,"is_alcohol":false},{"id":69818,"title":"Супы","logo":null,"description":"","restaurant":25,"parent":0,"position":6,"is_active":1,"food_count":3,"is_alcohol":false},{"id":69809,"title":"Пицца","logo":null,"description":"","restaurant":25,"parent":0,"position":7,"

In [100]:
categories = json.loads(categories)
categories

[{'id': 69813,
  'title': 'Роллы',
  'logo': None,
  'description': '',
  'restaurant': 25,
  'parent': 0,
  'position': 1,
  'is_active': 1,
  'food_count': 40,
  'is_alcohol': False},
 {'id': 69812,
  'title': 'Роллы теплые',
  'logo': None,
  'description': '',
  'restaurant': 25,
  'parent': 0,
  'position': 2,
  'is_active': 1,
  'food_count': 12,
  'is_alcohol': False},
 {'id': 69814,
  'title': 'Ассорти',
  'logo': None,
  'description': '',
  'restaurant': 25,
  'parent': 0,
  'position': 3,
  'is_active': 1,
  'food_count': 11,
  'is_alcohol': False},
 {'id': 69817,
  'title': 'Нигири суши',
  'logo': None,
  'description': '',
  'restaurant': 25,
  'parent': 0,
  'position': 4,
  'is_active': 1,
  'food_count': 5,
  'is_alcohol': False},
 {'id': 69811,
  'title': 'Гункан суши',
  'logo': None,
  'description': '',
  'restaurant': 25,
  'parent': 0,
  'position': 5,
  'is_active': 1,
  'food_count': 11,
  'is_alcohol': False},
 {'id': 69818,
  'title': 'Супы',
  'logo': None,


### Собираем продукты

In [103]:
products = requests.get(FOOD_API).text
products

'[{"id":688827,"title":"Маргарита 35см","logo":"https://thumb.chocofood.kz/Al9PaohjDHIFzoj6fix3B1j_BZk=/fit-in/640x640/https://chocofood.kz/media/site/restaurants/food/logo/2020/08/06/image_8278fed0-d2c8-46e0-a82e-b046ed282216.png","food_type":69809,"description":"Помидор, сыр моцарелла, орегано, базилик","ingredients":"Помидор, сыр моцарелла, орегано, базилик","price":1449.0,"discount_price":1449.0,"position":1,"is_active":1,"note":"","specialty":"","is_alcohol":false,"has_modification":true},{"id":688870,"title":"Гункан запеченный с крабом","logo":"https://thumb.chocofood.kz/5ocJrOwkjW3shpT_bPo-yPMUOI0=/fit-in/640x640/https://chocofood.kz/media/site/restaurants/food/logo/2020/09/15/food_62ip44O.png","food_type":69811,"description":"1 шт. Рис Японский, краб, сыр, соус Хот, водоросли Нори","ingredients":"1 шт. Рис Японский, краб, сыр, соус Хот, водоросли Нори","price":299.0,"discount_price":299.0,"position":1,"is_active":1,"note":"","specialty":"","is_alcohol":false,"has_modification":

In [104]:
products = json.loads(products)
products

[{'id': 688827,
  'title': 'Маргарита 35см',
  'logo': 'https://thumb.chocofood.kz/Al9PaohjDHIFzoj6fix3B1j_BZk=/fit-in/640x640/https://chocofood.kz/media/site/restaurants/food/logo/2020/08/06/image_8278fed0-d2c8-46e0-a82e-b046ed282216.png',
  'food_type': 69809,
  'description': 'Помидор, сыр моцарелла, орегано, базилик',
  'ingredients': 'Помидор, сыр моцарелла, орегано, базилик',
  'price': 1449.0,
  'discount_price': 1449.0,
  'position': 1,
  'is_active': 1,
  'note': '',
  'specialty': '',
  'is_alcohol': False,
  'has_modification': True},
 {'id': 688870,
  'title': 'Гункан запеченный с крабом',
  'logo': 'https://thumb.chocofood.kz/5ocJrOwkjW3shpT_bPo-yPMUOI0=/fit-in/640x640/https://chocofood.kz/media/site/restaurants/food/logo/2020/09/15/food_62ip44O.png',
  'food_type': 69811,
  'description': '1 шт. Рис Японский, краб, сыр, соус Хот, водоросли Нори',
  'ingredients': '1 шт. Рис Японский, краб, сыр, соус Хот, водоросли Нори',
  'price': 299.0,
  'discount_price': 299.0,
  'pos

### Конвертируем данные в pandas

In [125]:
"""
Категории
"""

df_categories = pd.DataFrame(categories)
df_categories = df_categories[['id', 'title']]
df_categories.head()

,id,title
0,69813,Роллы
1,69812,Роллы теплые
2,69814,Ассорти
3,69817,Нигири суши
4,69811,Гункан суши


In [126]:
"""
Продукты
"""

df_products = pd.DataFrame(products)
df_products = df_products[['id', 'title', 'food_type', 'description', 'price', 'logo']]
df_products.head()

,id,title,food_type,description,price,logo
0,688827,Маргарита 35см,69809,"Помидор, сыр моцарелла, орегано, базилик",1449.0,https://thumb.chocofood.kz/Al9PaohjDHIFzoj6fix...
1,688870,Гункан запеченный с крабом,69811,"1 шт. Рис Японский, краб, сыр, соус Хот, водор...",299.0,https://thumb.chocofood.kz/5ocJrOwkjW3shpT_bPo...
2,688875,Гункан Икура,69811,1 шт. Икра лосося,549.0,https://thumb.chocofood.kz/kZEcDASWmX_Q7e1lnUj...
3,688876,Гункан острый с креветкой,69811,"1 шт. Креветка,\r\nострый соус",299.0,https://thumb.chocofood.kz/zEJxaC5phvTMRswcO9G...
4,688879,Гункан острый с тунцом,69811,"1 шт. Тунец,\r\nострый соус",299.0,https://thumb.chocofood.kz/q5x9FfeuPwi5T7rFeNm...


In [127]:
"""
Соединяем категории и продукты
"""

df = df_products.merge(df_categories, left_on='food_type', right_on='id')
df.drop(['id_x', 'id_y', 'food_type'], axis=1, inplace=True)
df = df[['title_x', 'title_y', 'description', 'price', 'logo']]
df.columns = ['product_name', 'category', 'description', 'price', 'logo']
df.head()

,product_name,category,description,price,logo
0,Маргарита 35см,Пицца,"Помидор, сыр моцарелла, орегано, базилик",1449.0,https://thumb.chocofood.kz/Al9PaohjDHIFzoj6fix...
1,Пицца с угрем 35см,Пицца,"Угорь, брокколи, сливки, красный лук",1899.0,https://thumb.chocofood.kz/B-UPIjbqUNdTY1WizZI...
2,Пицца цезарь 35см,Пицца,"Куриное филе обжаренное, сыр моцарелла,\r\nсоу...",1749.0,https://thumb.chocofood.kz/pTlWRh7cyj5ZY6q_Mk0...
3,Морская 35см,Пицца,"Лосось, помидорки черри, фирменный соус,\r\nса...",1749.0,https://thumb.chocofood.kz/vkWiUqivOeC1wa1KioW...
4,4 сезона 30см,Пицца,"Салями, шампиньоны, брокколи, фарш, сыр\r\nмоц...",1499.0,https://thumb.chocofood.kz/_TtV9P1kCR-b9Tj64HN...


### Сохраняем картинки

In [146]:
def get_image(url:str, name:str) -> str:
    """
    На вход принимаем url и название продукта
    """
    
    if url:
        # Парсим url
        new_url = re.search('https://[a-zA-z0-9\.\/\-]+\.\w+$', url).group(0)
        
        # Скачиваем картинку
        img = requests.get(new_url).content
        
        # Узнаем расширение файла
        ext = re.search('\.\w+$', new_url).group(0)
        
        # Создаем новое имя для картинки из названия продукта и timestamp'а
        img_name = f"{name.lower()}_{datetime.now().strftime('%Y%m%d%H%M%S%f')}{ext}"
        
        # Сохраняем файл
        with open(f'chocofood_images/{img_name}', 'wb') as file:
            file.write(img)
        
        return img_name
    
    return None

In [147]:
df['logo'] = df.apply(lambda x: get_image(x.logo, x.product_name), axis=1)
df.head()

,product_name,category,description,price,logo
0,Маргарита 35см,Пицца,"Помидор, сыр моцарелла, орегано, базилик",1449.0,маргарита 35см_20201107142634241597.png
1,Пицца с угрем 35см,Пицца,"Угорь, брокколи, сливки, красный лук",1899.0,пицца с угрем 35см_20201107142634389872.png
2,Пицца цезарь 35см,Пицца,"Куриное филе обжаренное, сыр моцарелла,\r\nсоу...",1749.0,пицца цезарь 35см_20201107142634529101.png
3,Морская 35см,Пицца,"Лосось, помидорки черри, фирменный соус,\r\nса...",1749.0,морская 35см_20201107142634771453.png
4,4 сезона 30см,Пицца,"Салями, шампиньоны, брокколи, фарш, сыр\r\nмоц...",1499.0,4 сезона 30см_20201107142634914923.png


# Парсим полнеценный API

Получаем данные о погоде в Яндексе

In [148]:
"""
Для использования API Яндекса нужно создать ключ в кабинете разработчика
"""
API_KEY = "f1b7caeb-e80b-4a16-b277-87fcab42ca0d"

## Документация API

* https://yandex.ru/dev/weather/doc/dg/concepts/forecast-info.html

## Формат запроса

Населенный пункт указывается при помощи значений широты (параметр lat) и долготы (параметр lon). Если данные параметры заданы некорректно, ответ будет содержать код ошибки 404.

<code>GET https://api.weather.yandex.ru/v2/informers?
 lat=<широта>
 & lon=<долгота>
 & [lang=<язык ответа>]
X-Yandex-API-Key: <значение ключа></code>

## Получаем данные

In [152]:
YANDEX_URL = "https://api.weather.yandex.ru/v2/informers"
LAT = 43.254772
LON = 76.943244

### Готовим заголовки

In [149]:
headers = {
    'X-Yandex-API-Key': API_KEY
}

### Отправляем запрос

In [154]:
data = requests.get(f"{YANDEX_URL}?lat={LAT}&lon={LON}", headers=headers).text
data

'{"now":1604738708,"now_dt":"2020-11-07T08:45:08.728Z","info":{"lat":43.254772,"lon":76.943244,"url":"https://yandex.ru/pogoda/?lat=43.254772&lon=76.943244"},"fact":{"temp":16,"feels_like":12,"icon":"skc_d","condition":"clear","wind_speed":2,"wind_gust":4.5,"wind_dir":"ne","pressure_mm":700,"pressure_pa":934,"humidity":19,"daytime":"d","polar":false,"season":"autumn","obs_time":1604737875},"forecast":{"date":"2020-11-07","date_ts":1604685600,"week":45,"sunrise":"07:35","sunset":"17:36","moon_code":3,"moon_text":"moon-code-3","parts":[{"part_name":"evening","temp_min":9,"temp_max":12,"temp_avg":11,"feels_like":7,"icon":"skc_n","condition":"clear","daytime":"n","polar":false,"wind_speed":1.9,"wind_gust":5.4,"wind_dir":"s","pressure_mm":700,"pressure_pa":934,"humidity":36,"prec_mm":0,"prec_period":360,"prec_prob":0},{"part_name":"night","temp_min":8,"temp_max":9,"temp_avg":9,"feels_like":5,"icon":"skc_n","condition":"clear","daytime":"n","polar":false,"wind_speed":2.1,"wind_gust":6.5,"win

In [155]:
data = json.loads(data)
data

{'now': 1604738708,
 'now_dt': '2020-11-07T08:45:08.728Z',
 'info': {'lat': 43.254772,
  'lon': 76.943244,
  'url': 'https://yandex.ru/pogoda/?lat=43.254772&lon=76.943244'},
 'fact': {'temp': 16,
  'feels_like': 12,
  'icon': 'skc_d',
  'condition': 'clear',
  'wind_speed': 2,
  'wind_gust': 4.5,
  'wind_dir': 'ne',
  'pressure_mm': 700,
  'pressure_pa': 934,
  'humidity': 19,
  'daytime': 'd',
  'polar': False,
  'season': 'autumn',
  'obs_time': 1604737875},
 'forecast': {'date': '2020-11-07',
  'date_ts': 1604685600,
  'week': 45,
  'sunrise': '07:35',
  'sunset': '17:36',
  'moon_code': 3,
  'moon_text': 'moon-code-3',
  'parts': [{'part_name': 'evening',
    'temp_min': 9,
    'temp_max': 12,
    'temp_avg': 11,
    'feels_like': 7,
    'icon': 'skc_n',
    'condition': 'clear',
    'daytime': 'n',
    'polar': False,
    'wind_speed': 1.9,
    'wind_gust': 5.4,
    'wind_dir': 's',
    'pressure_mm': 700,
    'pressure_pa': 934,
    'humidity': 36,
    'prec_mm': 0,
    'prec_per

## Разбираем ответ

In [162]:
info = data.pop('info')
fact = data.pop('fact')
forecast = data.pop('forecast')
forecast_parts = forecast.pop('parts')

In [164]:
print('Data')
print(data)
print('')
print('Info')
print(info)
print('')
print('Fact')
print(fact)
print('')
print('Forecast')
print(forecast)
print('')
print('Forecast Parts')
print(forecast_parts)

Data
{'now': 1604738708, 'now_dt': '2020-11-07T08:45:08.728Z'}

Info
{'lat': 43.254772, 'lon': 76.943244, 'url': 'https://yandex.ru/pogoda/?lat=43.254772&lon=76.943244'}

Fact
{'temp': 16, 'feels_like': 12, 'icon': 'skc_d', 'condition': 'clear', 'wind_speed': 2, 'wind_gust': 4.5, 'wind_dir': 'ne', 'pressure_mm': 700, 'pressure_pa': 934, 'humidity': 19, 'daytime': 'd', 'polar': False, 'season': 'autumn', 'obs_time': 1604737875}

Forecast
{'date': '2020-11-07', 'date_ts': 1604685600, 'week': 45, 'sunrise': '07:35', 'sunset': '17:36', 'moon_code': 3, 'moon_text': 'moon-code-3'}

Forecast Parts
[{'part_name': 'evening', 'temp_min': 9, 'temp_max': 12, 'temp_avg': 11, 'feels_like': 7, 'icon': 'skc_n', 'condition': 'clear', 'daytime': 'n', 'polar': False, 'wind_speed': 1.9, 'wind_gust': 5.4, 'wind_dir': 's', 'pressure_mm': 700, 'pressure_pa': 934, 'humidity': 36, 'prec_mm': 0, 'prec_period': 360, 'prec_prob': 0}, {'part_name': 'night', 'temp_min': 8, 'temp_max': 9, 'temp_avg': 9, 'feels_like'

## Создаем датафреймы

### Текущий

In [169]:
general = {}
general.update(data)
general.update(info)
general.update(fact)
general.update(forecast)
general

{'now': 1604738708,
 'now_dt': '2020-11-07T08:45:08.728Z',
 'lat': 43.254772,
 'lon': 76.943244,
 'url': 'https://yandex.ru/pogoda/?lat=43.254772&lon=76.943244',
 'temp': 16,
 'feels_like': 12,
 'icon': 'skc_d',
 'condition': 'clear',
 'wind_speed': 2,
 'wind_gust': 4.5,
 'wind_dir': 'ne',
 'pressure_mm': 700,
 'pressure_pa': 934,
 'humidity': 19,
 'daytime': 'd',
 'polar': False,
 'season': 'autumn',
 'obs_time': 1604737875,
 'date': '2020-11-07',
 'date_ts': 1604685600,
 'week': 45,
 'sunrise': '07:35',
 'sunset': '17:36',
 'moon_code': 3,
 'moon_text': 'moon-code-3'}

In [174]:
df_general = pd.DataFrame.from_dict([general])
df_general

,now,now_dt,lat,lon,url,temp,feels_like,icon,condition,wind_speed,...,polar,season,obs_time,date,date_ts,week,sunrise,sunset,moon_code,moon_text
0,1604738708,2020-11-07T08:45:08.728Z,43.254772,76.943244,https://yandex.ru/pogoda/?lat=43.254772&lon=76...,16,12,skc_d,clear,2,...,False,autumn,1604737875,2020-11-07,1604685600,45,07:35,17:36,3,moon-code-3


In [188]:
df_general['now'] = df_general['now'].apply(lambda x: datetime.fromtimestamp(x))
df_general['obs_time'] = df_general['obs_time'].apply(lambda x: datetime.fromtimestamp(x))

In [192]:
df_general.drop(['now_dt', 'url', 'icon', 'date', 'date_ts'], axis=1, inplace=True)
df_general.set_index('now', inplace=True)

In [193]:
df_general

,lat,lon,temp,feels_like,condition,wind_speed,wind_gust,wind_dir,pressure_mm,pressure_pa,humidity,daytime,polar,season,obs_time,week,sunrise,sunset,moon_code,moon_text
now,,,,,,,,,,,,,,,,,,,,
2020-11-07 14:45:08,43.254772,76.943244,16,12,clear,2,4.5,ne,700,934,19,d,False,autumn,2020-11-07 14:31:15,45,07:35,17:36,3,moon-code-3


### Прогноз

In [213]:
df_forecast = pd.DataFrame(forecast_parts)
df_forecast

,part_name,temp_min,temp_max,temp_avg,feels_like,icon,condition,daytime,polar,wind_speed,wind_gust,wind_dir,pressure_mm,pressure_pa,humidity,prec_mm,prec_period,prec_prob
0,evening,9,12,11,7,skc_n,clear,n,False,1.9,5.4,s,700,934,36,0,360,0
1,night,8,9,9,5,skc_n,clear,n,False,2.1,6.5,s,699,932,30,0,360,0


In [214]:
df_forecast.drop(['icon'], axis=1, inplace=True)
df_forecast['now'] = df_general.iloc[0].name
df_forecast

,part_name,temp_min,temp_max,temp_avg,feels_like,condition,daytime,polar,wind_speed,wind_gust,wind_dir,pressure_mm,pressure_pa,humidity,prec_mm,prec_period,prec_prob,now
0,evening,9,12,11,7,clear,n,False,1.9,5.4,s,700,934,36,0,360,0,2020-11-07 14:45:08
1,night,8,9,9,5,clear,n,False,2.1,6.5,s,699,932,30,0,360,0,2020-11-07 14:45:08
